In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
import os

In [3]:
# Accedemos a cada train y a cada test tanto de X como de Y, y los dejamos metidos en variables para el siguiente paso!! 
# Los separamos en base path y el resto de la ruta de acceso para tener el código mas legible. 
current_dir = os.getcwd()
BASE_PATH = os.path.join(current_dir, "../data/csv_model")
TRAIN_PATHS = [
    "X_train_con_outliers.csv",
    "X_train_sin_outliers.csv"
]

# Concatenamos el base path al resto de la ruta de acceso para tener el código completo dentro de la variable que usaremos despues. 
TRAIN_DATASETS = []
for path in TRAIN_PATHS:
    TRAIN_DATASETS.append(
        # pd.read_excel(BASE_PATH + "/" + path)
        pd.read_csv(f"{BASE_PATH}/{path}") 
        # pd.read_excel(os.path.join(BASE_PATH, path))
    )

# Los separamos en base path y el resto de la ruta de acceso para tener el código mas legible. 
TEST_PATHS = [
    "X_test_con_outliers.csv",
    "X_test_sin_outliers.csv"
]

# Concatenamos el base path al resto de la ruta de acceso para tener el código completo dentro de la variable que usaremos despues. 
TEST_DATASETS = []
for path in TEST_PATHS:
    TEST_DATASETS.append(
        pd.read_csv(f"{BASE_PATH}/{path}")
    )

# Concatenamos el base path al resto de la ruta de acceso para tener el código completo dentro de la variable que usaremos despues. 
y_train = pd.read_csv(f"{BASE_PATH}/y_train.csv")
y_test = pd.read_csv(f"{BASE_PATH}/y_test.csv")

#### Boosting with `XGBClassifier` and `GridSearchCV`

In [4]:
from xgboost import XGBRegressor
from sklearn.metrics import r2_score

# Le pasamos los train y los test al modelo , lo entrenamos y metemos los resultados del test en una lista para poder comparar y escoger el mejor. 

results = []

for index, dataset in enumerate(TRAIN_DATASETS):
    print(index)
    model = XGBRegressor(random_state = 77, device='cuda')
    
    model.fit(dataset, y_train)
    y_pred_train = model.predict(dataset)
    y_pred_test = model.predict(TEST_DATASETS[index])

    results.append(
        {
            "train": r2_score(y_train, y_pred_train),
            "test": r2_score(y_test, y_pred_test)
        }
    )

results

0
1


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [12:35:34] WARNING: /Users/runner/work/xgboost/xgboost/src/context.cc:203: XGBoost is not compiled with CUDA support.
  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [12:35:34] WARNING: /Users/runner/work/xgboost/xgboost/src/context.cc:203: XGBoost is not compiled with CUDA support.
  bst.update(dtrain, iteration=i, fobj=obj)


[{'train': 0.8018825054168701, 'test': 0.3218003511428833},
 {'train': 0.8018825054168701, 'test': 0.3218003511428833}]

In [5]:
# Optimización del modelo. 

best_dataset = 0 # Porque de results, el mayor acuracy en train y test.

# Ponemos los hyperparametros según la documentación.  

hyperparams = {
    'n_estimators': [100, 150],  # Número de árboles (estimadores)
    'learning_rate': [0.05, 0.1],  # Tasa de aprendizaje
    'max_depth': [3, 5, 10],  # Profundidad máxima de los árboles
    'subsample': [0.8, 1.0],  # Fracción de muestras por árbol
    'colsample_bytree': [0.8, 1.0],  # Fracción de características por árbol
    'gamma': [0.2, 0.5],  # Parámetro de regularización
    'min_child_weight': [1, 5],  # Peso mínimo de las muestras para dividir un nodo
}

from sklearn.model_selection import GridSearchCV 

model = XGBRegressor(random_state = 77) # Volvemos a seleccionar el modelo sin entrenar. 
grid = GridSearchCV(model, hyperparams, scoring = "r2") # Metemos en el gris los nuevos hyperparametros

In [6]:
grid.fit(TRAIN_DATASETS[best_dataset], y_train)

GridSearchCV(estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, feature_weights=None,
                                    gamma=None, grow_policy=None,
                                    importance_type=None,
                                    interaction_constraints=None,
                                    lear...
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None, ...),
             param_grid={'colsample_bytree': [0.8, 1.0], 'gamma': [0.2, 0.5],
                         'learning_rate': [0.05, 0.1], 'max_depth': [3, 5, 10],
                         'min_child_weight': [1, 5], 'n_estimators': [100, 150],
                         'subsample': [0.8, 1.0]},
             scoring='r2')

In [7]:
grid.best_estimator_

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=0.2, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.05, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
             max_leaves=None, min_child_weight=5, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=100,
             n_jobs=None, num_parallel_tree=None, ...)

In [8]:
# Predecimos con el nuevo modelo ya entrenado por los unevos parametros.  

final_model = grid.best_estimator_
y_pred_train = final_model.predict(TRAIN_DATASETS[best_dataset])
y_pred_test = final_model.predict(TEST_DATASETS[best_dataset])

In [9]:
score_train = r2_score(y_train, y_pred_train),
score_test = r2_score(y_test, y_pred_test)

print(f'Score train: {score_train}')
print(f'Score test: {score_test}')

Score train: (0.37658244371414185,)
Score test: 0.35302889347076416


In [10]:
# import pickle

# with open("/Users/julian/Desktop/vs code/Boosting_algorithm_model_for_machine_learning_Project_18_Julian_Lopez/Boosting_algorithm_model_for_machine_learning_Project_18_Julian_Lopez/models/Boosting_with_XGBClassifier_and_GridSearchCV_model.sav", "wb") as file:
#     pickle.dump(final_model, file)

#### MAE

In [11]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

results = []

for index, dataset in enumerate(TRAIN_DATASETS):
    print(index)
    model = XGBRegressor(random_state=77, device='cuda')
    
    model.fit(dataset, y_train)
    y_pred_train = model.predict(dataset)
    y_pred_test = model.predict(TEST_DATASETS[index])

    results.append(
        {
            "train_mae": mean_absolute_error(y_train, y_pred_train),
            "test_mae": mean_absolute_error(y_test, y_pred_test)
        }
    )

results


0
1


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [12:37:46] WARNING: /Users/runner/work/xgboost/xgboost/src/context.cc:203: XGBoost is not compiled with CUDA support.
  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [12:37:46] WARNING: /Users/runner/work/xgboost/xgboost/src/context.cc:203: XGBoost is not compiled with CUDA support.
  bst.update(dtrain, iteration=i, fobj=obj)


[{'train_mae': 0.8743097186088562, 'test_mae': 1.6540964841842651},
 {'train_mae': 0.8743097186088562, 'test_mae': 1.6540964841842651}]